# 09: Building Tables and Reports

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/09_tables.ipynb)

Forecast bulletins, QA checks, and research briefs often rely on tidy tables. This notebook shows how to transform xsnow datasets into flexible tabular outputs.

## What You'll Learn

- Flattening xsnow datasets into pandas DataFrames
- Working with multi-index structures efficiently
- Creating pivot tables and summary statistics
- Joining xsnow data with external lookup tables
- Exporting tables to CSV and Markdown formats


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [ ]:
%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow


In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import xsnow

sns.set(style='whitegrid', context='talk')


In [ ]:
        print("Loading xsnow sample data for tabular analysis...")
        print("Using xsnow.single_profile_timeseries()")
        print()

        try:
            ds = xsnow.single_profile_timeseries()
            base_ds = getattr(ds, 'data', ds)
            print("✅ Data loaded successfully!")
            print(base_ds)
        except Exception as exc:
            print(f"❌ Error loading sample data: {exc}")
            print("
Make sure xsnow is properly installed:")
            print("  pip install git+https://gitlab.com/avacollabra/postprocessing/xsnow")
            ds = None
            base_ds = None


## Part 1: Flatten to a DataFrame

Convert the multi-dimensional dataset into a tidy table for further analysis.


In [ ]:
        if base_ds is not None:
            df = base_ds.to_dataframe().reset_index()
            print(df.head())
            print("
Columns:", df.columns.tolist())
        else:
            df = None
            print("Dataset not loaded. Restart the notebook if needed.")


## Part 2: Optimize Multi-Index Operations

Use pandas indexes to filter and aggregate efficiently across dimensions.


In [ ]:
if df is not None:
    multi_df = base_ds.to_dataframe()
    # Example: average density by time and aspect (slope)
    if 'density' in multi_df.columns:
        grouped = multi_df['density'].groupby(['time', 'slope']).mean().unstack('slope')
        display(grouped.head())
    else:
        print("Density variable not available in this dataset.")


## Part 3: Build Pivot Tables for Reports

Pivot tables highlight metrics such as layer counts per grain type or daily minimum temperatures.


In [ ]:
if df is not None:
    if 'grain_type' in df.columns:
        pivot = pd.pivot_table(
            df,
            values='density' if 'density' in df.columns else None,
            index='time',
            columns='grain_type',
            aggfunc='count',
            fill_value=0
        )
        display(pivot.head())
    else:
        print("No 'grain_type' column found. Adjust the pivot to match your dataset.")


## Part 4: Join with External Lookup Tables

Merge xsnow data with reference metadata, such as hazard ratings or instrument calibration notes.


In [ ]:
if df is not None:
    # Mock hazard rating lookup
    hazard_lookup = pd.DataFrame({
        'time': pd.to_datetime(df['time'].unique()).sort_values(),
        'hazard_rating': np.random.choice(['Low', 'Moderate', 'Considerable'], size=df['time'].nunique()),
    })
    merged = df.merge(hazard_lookup, on='time', how='left')
    display(merged[['time', 'hazard_rating']].drop_duplicates().head())


## Part 5: Export Tables

Save tables to CSV or Markdown to share with teammates.


In [ ]:
if df is not None:
    output_table = df[['time', 'layer', 'density']].head(20) if 'density' in df.columns else df.head(20)
    csv_path = 'sample_table.csv'
    md_path = 'sample_table.md'
    output_table.to_csv(csv_path, index=False)
    output_table.to_markdown(md_path, index=False)
    print(f"Wrote {csv_path} and {md_path} with sample outputs.")


## Summary

- xsnow datasets convert cleanly to pandas DataFrames for tabular work.
- Multi-index groupby operations power efficient aggregations.
- Export results as CSV or Markdown to integrate with bulletins and reports.

**Next steps:** Continue with `09a_tables_reporting_templates.ipynb` for reusable reporting workflows.
